In [1]:
from tqdm import tqdm
import json

from videograph import VideoGraph
from utils.general import *
from utils.video_processing import *
from utils.chat_api import *
from prompts import *

from retrieve import answer_with_retrieval
from generate_memory import streaming_process_video

processing_config = json.load(open("configs/processing_config.json"))
memory_config = json.load(open("configs/memory_config.json"))

/usr/local/lib/python3.9/dist-packages/bytedmetrics/__init__.py:10: UserWarning: bytedmetrics is renamed to bytedance.metrics, please using `bytedance.metrics` instead of `bytedmetrics`
  warnings.warn("bytedmetrics is renamed to bytedance.metrics, please using `bytedance.metrics` instead of `bytedmetrics`")
/home/tiger/.local/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
/usr/local/lib/python3.9/dist-packages/bytedtrace/__init__.py:108: UserWarning: [bytedtrace] global tracer is already initialized.
  warnings.warn('[bytedtrace] global tracer is already initialized.')


In [2]:
# video paths can be paths to directories or paths to mp4 files
video_paths = ["/mnt/hdfs/foundation/longlin.kylin/mmagent/data/test_video_clips/test2"]
save_dir = processing_config["save_dir"]
max_workers = 1


def process_single_video(video_path):
    video_graph = VideoGraph(**memory_config)
    streaming_process_video(video_graph, video_path)


# Process videos in parallel using ThreadPoolExecutor with max_workers limit
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Process videos in parallel using map
    list(
        tqdm(
            executor.map(process_single_video, video_paths),
            total=len(video_paths),
            desc="Processing videos",
        )
    )

Processing videos:   0%|          | 0/1 [00:00<?, ?it/s]

Starting processing 0-th (out of 5) clip: /mnt/hdfs/foundation/longlin.kylin/mmagent/data/test_video_clips/test2/1.mp4
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.1.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': 1278, 'fps': 23.976023976023978, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.3.100 libx264'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 129, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 30.0, 'bitrate': 1413, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_numb

100%|██████████| 38/38 [00:17<00:00,  2.13it/s]


Write face detection results to /mnt/hdfs/foundation/longlin.kylin/mmagent/data/intermediate_outputs/test2_30_5_-1_10_20_0.3_0.6/clip_0_faces.json
Finish processing faces
No qualified faces detected


Processing videos:   0%|          | 0/1 [00:41<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
video_graph_path = (
    "data/mems/5-Poor-People-vs-1-Secret-Millionaire_60_5_-1_10_20_0.3_0.6.pkl"
)
video_graph = load_video_graph(video_graph_path)

question = "How might the social development of children living in military families be affected?"

answer = answer_with_retrieval(
    video_graph,
    question,
    query_num=processing_config["query_num"],
    topk=processing_config["topk"],
    mode="argmax",
)

In [ ]:
video_graph_path = (
    "data/mems/5-Poor-People-vs-1-Secret-Millionaire_60_5_-1_10_20_0.3_0.6.pkl"
)
video_graph = load_video_graph(video_graph_path)
video_graph.visualize()

# print all episodic and semantic nodes
for node_id in video_graph.text_nodes:
    print(video_graph.nodes[node_id].metadata["contents"][0])